# 수집된 뉴스 기사 및 댓글에 대한 감정 분석
## * Doc2Vec 
* 데이터 
> 2017년 12월 1일부터 2018년 2월 1일까지 63일간 [네이버](http://www.naver.com)와 [다음](http://www.daum.net)의 랭킹뉴스와 뉴스의 댓글을 크롤링함.

In [1]:
import pickle
import html
import multiprocessing
from collections import namedtuple, OrderedDict
import re
import sys
import os
from glob import glob
import warnings

os.environ['KERAS_BACKEND']='tensorflow'

from numba import jit
import numpy as np
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
import pandas as pd

from gensim.models import doc2vec, KeyedVectors
from gensim.models.doc2vec import TaggedDocument

from konlpy.utils import pprint

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report, roc_curve,  accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import scale, MinMaxScaler
from sklearn.manifold import TSNE

import keras.backend.tensorflow_backend as K
from keras.preprocessing import sequence
from keras_tqdm import TQDMCallback, TQDMNotebookCallback
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.layers import Input, Flatten, Dense, Embedding, embeddings, merge, Dropout, Activation,  LSTM, Bidirectional, SimpleRNN, GRU
from keras.layers.convolutional import Conv1D, Conv2D
from keras.layers.pooling import MaxPooling1D, GlobalMaxPooling1D
from keras.layers.normalization import BatchNormalization
from keras.layers.core import SpatialDropout1D
from keras.utils import np_utils
from tensorflow.python.client import device_lib
from keras.layers.merge import dot

import xgboost as xgb

import matplotlib.pyplot as plt

import bokeh.plotting as bp
from bokeh.models import HoverTool, BoxSelectTool
from bokeh.plotting import figure, show, output_notebook

Using TensorFlow backend.


In [2]:
import Database_Handler as dh
import Basic_Module as bm

In [4]:
from ckonlpy.tag import Twitter
from konlpy.tag import Mecab
ct = Twitter()
mecab = Mecab()

## Stopwords

In [5]:
stopwords = open('./data/stopwordsList.txt',encoding='utf-8').readlines()
stopwords = list(map(lambda x: x.strip(), stopwords))

## Load Data

### Path

In [34]:
if sys.platform =='darwin':
    'daum_news_sentiment_analysis.csv'
    loadModelPath = '/Volumes/disk1/model/'
    classifierPath = '/Volumes/disk1/data/pre_data/classifier/'
    news_senti_outcome = '/Volumes/disk1/outcome_for_News_sentiment_analysis/'
    daumCommentPath = './data/daum_Comments/'
    naverCommentsPath = './data/naver_Comments/'
elif sys.platform =='win32':
    loadModelPath = 'd:/model/'
    classifierPath = 'd:/data/pre_data/classifier/'
    newsPath = './data/pre_data/news_sentiment/'
    news_senti_outcome = './outcome_for_News_sentiment_analysis/'
    daumCommentPath = './data/daum_Comments/'
    naverCommentsPath = './data/naver_Comments/'

### News

In [7]:
os.listdir(news_senti_outcome)

['naver_news_sentiment_analysis.csv', 'daum_news_sentiment_analysis.csv']

In [8]:
# Naver
naverData = pd.read_csv(os.path.join(news_senti_outcome, 'naver_news_sentiment_analysis.csv'), index_col=0, header= 0, encoding = 'utf-8')
naverData['site'] = ['Naver'] * naverData.shape[0]
#naverData.head()

In [9]:
naverData[naverData.number_of_crawled_comment == 0]

,category,date,press,number_of_comment,number_of_crawled_comment,rank,title,mainText,keywords,extracted_keywords,negative,positive,Decision,site
id,,,,,,,,,,,,,,
5a32e268588c136a189e66d0,IT/과학,2017.12.14,노컷뉴스,1,0,28,"드론, 하천측량 업무에 본격 활용된다","국토부, 드론기반 하상변동조사·하천측량 시범사업 착수 [CBS노컷뉴스 곽영식 기자...","['드론', '시범사업', '국토교통부']","{'기존', '활용', '드론', '시범 사업', '국토부', '하천 측량'}",28.0,80.0,positive,Naver
5a44bcf3588c13269cc40b83,생활/문화,2017.12.27,연합뉴스,0,0,29,임실 변천사 한눈에…1730년 작성한 운수지 공개,(임실=연합뉴스) 이윤승 기자 = 1730년대 전북 임실군의 변천사를 한눈에 파악할...,"['조선시대', '달리']","{'중요한', '작성', '참여', '운수', '설명', '확보', '임실군', '...",38.0,70.0,positive,Naver
5a6668ee588c131c881e0a35,IT/과학,2018.01.19,뉴스1,0,0,21,CJ헬로 매각 부인한 CJ오쇼핑 '3개월내 매각못한다'…왜?,© News1 안은나 기자 계속 제기되는 매각설에 CJ오쇼핑과 CJ헬로 '곤혹'...,"['이슈 · LGU+, CJ헬로 인수설']","{'CJ 헬로', '공시', '결정', 'CJ오쇼핑', '매각설'}",85.0,23.0,negative,Naver


naver 뉴스를 크롤링한 데이터베이스에서는 댓글수가 0개인 뉴스가 3개 존재함

#### 크롤링된 댓글수가 0개인 것을 찾아보자

In [10]:
# Daum
daumData = pd.read_csv(os.path.join(news_senti_outcome, 'daum_news_sentiment_analysis.csv'), index_col=0, header= 0, encoding = 'utf-8')
daumData['site'] = ['daum'] * daumData.shape[0]
#daumData.head()

#### 크롤링된 댓글수가 0개인 것을 찾아보자

In [11]:
daumData[daumData.number_of_crawled_comment == 0]

,category,date,press,number_of_comment,number_of_crawled_comment,rank,title,mainText,keywords,extracted_keywords,negative,positive,Decision,site
id,,,,,,,,,,,,,,
5a3c39e7588c132bc05289fb,뉴스,2017.12.21,노컷뉴스,17384,0,7,"[인터뷰] 서민 ""'문빠' 해악, 더 두고 볼 수 없었다""","- 낚싯배 사고 & 中 기자폭행 관련 '문빠' 반응, 도를 넘은 듯 - '보수 쪽도...","['정관용', '댓글', '낚싯배']","{'대통령', '정관용', '서민 교수', '낚싯배', '댓글', '문빠들', '폭행'}",95.0,13.0,negative,daum
5a64385d588c134738c38086,연예,2018.01.07,뉴스엔,0,0,45,"'언니네' 송은이 ""라이언전, 10년 전 박소현 맞선남으로 출연""",[뉴스엔 박수인 기자] 개그우먼 송은이가 작곡가 라이언전의 과거를 공개했다. 1월 ...,"['박소현', '언니네라디오', '김숙', '개그우먼', '게스트']","{'언니네', '민망', '방송', '박소현 맞선남', '송은이', '작곡가 라이언전'}",65.0,43.0,negative,daum
5a6472e8588c131b18d57abe,스포츠,2018.01.11,MK스포츠,0,0,38,"MLB 에이전트, 선수 샤워 장면 몰래 촬영해 '충격'",[매경닷컴 MK스포츠 김재호 기자] 다수의 메이저리그 선수를 고객으로 보유하고 있는...,NaN,"{'메이저리그', '에이전시', '회사', '사장', 'CSE', '샤워', '선수...",39.0,69.0,positive,daum


daum 뉴스를 크롤링한 데이터베이스에서는 댓글수가 0개인 뉴스가 3개 존재함.
> 뉴스 1건의 댓글수(글쓴이가 삭제한 댓글 포함)가 17,384건이지만, 크롤링된 댓글은 0개............  
>> 2017.12.21 뉴스. 발견은 2018년 3월 14일. 3달이나 지나서 댓글을 수집하면 삭제할 사람은 삭제했을 것. 
>> * 아쉽지만 데이터베이스에 댓글이 존재하는 것에 대해서만 진행하기로 함. 

### 새롭게 정의된 뉴스목록

In [18]:
reNaverData = naverData[naverData.number_of_crawled_comment != 0]
reDaumData = daumData[daumData.number_of_crawled_comment != 0]

In [19]:
reDaumData.head()

,category,date,press,number_of_comment,number_of_crawled_comment,rank,title,mainText,keywords,extracted_keywords,negative,positive,Decision,site
id,,,,,,,,,,,,,,
5a2a61bf588c13481c229d1e,뉴스,2017.12.07,세계일보,1093,903,1,"""밤이 무섭다""..비아그라 공장 연기에 남성들 부작용 호소","주민들은 공장에서 배출된 연기가 '남성이 매우 건강해지는 부작용'을 일으킨다며, ...","['부작용', '비아그라', '아일랜드']","{'지역', '남성들', '건강', '공장', '연기', '부작용', '세보 효과'}",50.0,58.0,positive,daum
5a2a61bf588c13481c229d1f,뉴스,2017.12.07,헬스조선,603,386,2,식후 커피·늦은 양치질..점심식사 후 하면 안 좋은 습관 3가지,점심식사를 마친 후 후식으로 커피를 마시는 사람들이 많다. 실제로 직장이 밀집돼 ...,"['커피', '낮잠', '음식물']","{'입냄새', '건강', '자세', '철분', '낮잠', '점심 식사', '치아',...",81.0,27.0,negative,daum
5a2a61bf588c13481c229d20,뉴스,2017.12.07,연합뉴스,1067,807,3,"'십년지기 생매장' 진짜 이유는..""'청부 통정' 알려질까 봐""",(성남=연합뉴스) 최해민 기자 = 십년지기 지인을 산 채로 묻어 살해한 50대 여...,"['살인혐의', '철원', '검찰송치']","{'진술', '경찰', '남편', '아들', '성관계', '주변', '지인', '철...",99.0,9.0,negative,daum
5a2a61bf588c13481c229d21,뉴스,2017.12.07,헤럴드경제,418,368,4,"신영자, 억 소리나는 갑질","신영자, 적용안된 혐의→검찰 상고에서 인정\n신영자, 얼마를 어떻게 받았나 [헤럴...","['신영자', '갑질', '롯데백화점']","{'롯데', '혐의', '매장', '검찰', '유통업체', '징역', '신영자 이사...",77.0,31.0,negative,daum
5a2a61bf588c13481c229d22,뉴스,2017.12.07,연합뉴스,434,367,5,"""배신하지마"" 20대女 살인 피의자 유치장서 공범 남친에 쪽지",(청주=연합뉴스) 이승민 기자 = 지난 9월 청주의 한 하천에서 20대 여성을 둔기...,"['공범', '살인', '과자']","{'과자', '혐의', '쪽지', '남자친구', '유치장', '경찰', '범행', ...",102.0,6.0,negative,daum


In [38]:
def Make_Comments_File(filepath, row):
    import Basic_Module as bm
    import os
    filename = row.name
    absPath = os.path.join(filepath, filename+'.csv')
    if os.path.isfile(absPath):
        pass
    else:
        comments = bm.Read_Comments(row)
        return comments
        #comments.to_csv(absPath, index=None, header=True, encoding='utf-8')
    
    

In [39]:
x = Make_Comments_File(daumCommentPath, x)

(903, 8)


In [40]:
x

,_id,category,comments,date,rank,site,공감,비공감
0,5a2a61c1588c13481c22a047,뉴스,#옵션열기는 누굽니까!! 윤석렬검사에 강장제와 꽃을 보내자!! 그리고 무일이... ...,2017.12.07,1,daum,0,0
1,5a2a61c1588c13481c22a0e9,뉴스,"'""매우 건강""ㅎㅎㅎㅎㅎㅎㅎ",2017.12.07,1,daum,0,0
2,5a2a61c1588c13481c229dde,뉴스,'남성이 매우 건강해지는 부작용'을 일으킨다며 ㅋㅋ 너무 건강해져도 탈이것네.. 시...,2017.12.07,1,daum,9,0
3,5a2a61c1588c13481c22a0f8,뉴스,.,2017.12.07,1,daum,0,0
4,5a2a61c1588c13481c22a13a,뉴스,1. 미국은 한반도 분단 책임지고 한국의 평화통일에 사심없이 적극협조해야 합니다. ...,2017.12.07,1,daum,0,3
5,5a2a61c1588c13481c22a04b,뉴스,20대 후반부터 습관적으로 비아그라 즐기던 친구 30대 후반에 발기부전 오더라.. ...,2017.12.07,1,daum,0,0
6,5a2a61c1588c13481c229e8b,뉴스,24시간 꼴려있나부네,2017.12.07,1,daum,1,0
7,5a2a61c1588c13481c229e4d,뉴스,24시간 꼴리면 죽을 맛이지?,2017.12.07,1,daum,1,0
8,5a2a61c1588c13481c22a11f,뉴스,24시간 풀발기 ㅜㅜ,2017.12.07,1,daum,0,1
9,5a2a61c1588c13481c229ed2,뉴스,24시간발기상태유지.,2017.12.07,1,daum,1,0


In [47]:
x[~x.comments.str.match('.*[0-9a-zA-Z가-힣ㄱ-하-ㅣ]+')]

,_id,category,comments,date,rank,site,공감,비공감
3,5a2a61c1588c13481c22a0f8,뉴스,.,2017.12.07,1,daum,0,0


## TaggedDocument For Doc2Vec

In [13]:
TaggedDocument = namedtuple('TaggedDocument', 'words tags sentiment')

## Doc2Vec Model

In [ ]:
model1 = doc2vec.Doc2Vec.load(loadModelPath+'doc2vec_size-1000_epoch-20_window-10_negative-7_hs-0_dm-1_dm_concat-0_dm_mean-1_by-ct.model')
name1 = '-'.join(re.split('[\(\),\/]',str(model1)))+taggerName
model2 = doc2vec.Doc2Vec.load(loadModelPath+'doc2vec_size-1000_epoch-20_window-5_negative-7_hs-0_dm-1_dm_concat-1_dm_mean-0_by-ct.model')
name2 = '-'.join(re.split('[\(\),\/]',str(model2)))+taggerName
model3 = doc2vec.Doc2Vec.load(loadModelPath+'doc2vec_size-1000_epoch-20_window-None_negative-7_hs-0_dm-0_dm_concat-0_dm_mean-0_by-ct.model')
name3 = '-'.join(re.split('[\(\),\/]',str(model3)))+taggerName